In [1]:
# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD,Adam
import keras
# Conv1D + LSTM
from keras.layers import Conv1D,MaxPooling1D
from keras.layers import LSTM
from keras.layers import Dense,Dropout
from keras.layers import Flatten

In [3]:
# Reading Data

In [4]:
col_list = ["sport","dport", 
            "pkts","bytes", 
            "state","dur", 
            "mean","stddev", 
            "sum","min", 
            "max","spkts", 
            "dpkts","sbytes", 
            "dbytes","rate", 
            "srate","drate",
            "attack"
           ]

In [6]:
df_num=pd.read_csv('./archive/ddos/DDoS_TCP.csv',sep=';',usecols=col_list)#nrows=50000,
print(df_num)

            sport  dport   pkts    bytes state        dur      mean    stddev  \
0          3456.0   80.0  10242  9972888   CON  75.066856  4.989200  0.006723   
1          8080.0   80.0  10246  9978140   CON  75.066925  4.989192  0.006743   
2            80.0   80.0   5116  4911913   INT  75.063446  4.990958  0.005609   
3            80.0   80.0   5117  4789010   CON  75.063515  4.990964  0.003873   
4           365.0  565.0   2753   165180   INT  75.240524  4.954834  0.048765   
...           ...    ...    ...      ...   ...        ...       ...       ...   
19548230  42343.0   80.0      1      154   REQ   0.000000  0.000000  0.000000   
19548231  42344.0   80.0      1      154   REQ   0.000000  0.000000  0.000000   
19548232  42345.0   80.0      1      154   REQ   0.000000  0.000000  0.000000   
19548233      NaN    NaN      5      300   CON   3.098834  3.098834  0.000000   
19548234      NaN    NaN      3      180   CON   1.032404  1.032404  0.000000   

                sum       m

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, metrics
cat_features = ['state']
encoder = LabelEncoder()
label_encoder = preprocessing.LabelEncoder()
for feature in cat_features:
    encoded = label_encoder.fit_transform(df_num[cat_features])
    df_num['state'] = encoded

C:\Users\SIDDHARTH SINGH\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
len(df_num)

19548235

In [ ]:
# Data Preparation

In [9]:
df_num.dtypes

sport     float64
dport     float64
pkts        int64
bytes       int64
state       int32
dur       float64
mean      float64
stddev    float64
sum       float64
min       float64
max       float64
spkts       int64
dpkts       int64
sbytes      int64
dbytes      int64
rate      float64
srate     float64
drate     float64
attack      int64
dtype: object

In [ ]:
### Removing nan values

In [10]:
print(df_num.isna().sum().sum())
df_num=df_num.dropna()
print(df_num.isna().sum().sum())

1022
0


In [11]:
df_num.sample(5)

,sport,dport,pkts,bytes,state,dur,mean,stddev,sum,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack
13044748,55038.0,80.0,1,154,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,154,0,0.000000,0.000000,0.0,1
7176901,43213.0,80.0,3,462,5,16.984003,0.000000,0.000000,0.000000,0.000000,0.000000,3,0,462,0,0.117758,0.117758,0.0,1
3512385,23714.0,80.0,2,308,5,3.819531,3.819531,0.000000,3.819531,3.819531,3.819531,2,0,308,0,0.261812,0.261812,0.0,1
7818116,21151.0,80.0,4,522,7,8.461223,1.578547,1.578547,3.157094,0.000000,3.157094,3,1,462,60,0.354559,0.236372,0.0,1
14488377,38812.0,80.0,4,616,5,51.924877,0.000000,0.000000,0.000000,0.000000,0.000000,4,0,616,0,0.057776,0.057776,0.0,1


In [ ]:
# Distribution Classes

In [12]:
df_num.groupby('attack').size()

attack
0         620
1    19547104
dtype: int64

In [ ]:
# Remove Useless Features

In [13]:
for col in (df_num.iloc[:,:-1].columns):
    if(df_num[col].min()==df_num[col].max()):
            df_num.drop(col, axis=1, inplace=True)

In [ ]:
# Feature Scaling

In [14]:
# Normalization OR Standardization
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/(df[col].std()) # Standardization
    #df[col]= (df[col]-df[col].min())/(df[col].max()-df[col].min()) #Normalization

for i in (df_num.iloc[:,:-1].columns):
    standardize (df_num,i)

df_num.head()

,sport,dport,pkts,bytes,state,dur,mean,stddev,sum,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack
0,-1.548194,-0.037286,675.736035,703.160651,-5.246894,3.957576,2.468846,-0.842947,20.190914,3.146913,1.296183,505.592497,786.258320,540.080391,805.656119,58.227191,119.466340,195.626713,0
1,-1.303736,-0.037286,676.000054,703.530979,-5.246894,3.957581,2.468840,-0.842925,20.190879,3.146903,1.296216,505.395001,787.180137,570.688614,762.784607,58.249946,119.419450,195.855940,0
2,-1.726673,-0.037286,337.396105,346.302181,-2.882854,3.957337,2.470041,-0.844178,20.198370,3.158983,1.296184,504.802514,-0.051547,543.889267,-0.005093,28.999581,119.284684,-0.021749,0
3,-1.726673,-0.037286,337.462110,337.636069,-5.246894,3.957342,2.470045,-0.846095,20.198394,3.158984,1.296184,504.901262,-0.051547,530.278685,-0.005093,29.005256,119.307963,-0.021749,0
4,-1.711606,0.261762,181.427074,11.601481,-2.882854,3.969784,2.445479,-0.796511,20.045182,3.044447,1.294951,271.461374,-0.051547,18.224300,-0.005093,15.487984,63.869466,-0.021749,0


In [15]:
target=['attack']
features = [c for c in df_num.columns if c!="attack"]

In [ ]:
# Split DataSet

In [16]:
X = df_num[features].values # Features
y = df_num[target].values # Target

X=X.astype(np.float32)
y=y.astype(np.float32)

X.shape,y.shape,

((19547724, 18), (19547724, 1))

In [17]:
del df_num

In [18]:
gc.collect()

415

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [20]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((15638179, 18), (3909545, 18), (15638179, 1), (3909545, 1))

In [21]:
X_train_cnn = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test_cnn = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
print(X_train_cnn.shape)
print(X_test_cnn.shape)


(15638179, 18, 1)
(3909545, 18, 1)


In [ ]:
# Build CNN Model

In [22]:
learning_rate=0.001
batch_size=5000
epochs = 15

model_save = ModelCheckpoint('./DDoS_TCP.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                           patience = 8, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.6, 
                              patience = 4, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)

In [23]:
X_test.shape[1]

18

In [ ]:
model = Sequential()
model.add(Conv1D(16,2,input_shape = (X_test.shape[1], 1)))
model.add(Conv1D(16,2))
model.add(MaxPooling1D(1))


model.add(Conv1D(32,2))
model.add(Conv1D(32,2))
model.add(MaxPooling1D(1))

model.add(Conv1D(64,2))
model.add(Conv1D(64,2))
model.add(MaxPooling1D(1))

model.add(Conv1D(128,2))
model.add(Conv1D(128,2))
model.add(MaxPooling1D(1))

#model.add(LSTM(32, activation = 'relu', return_sequences=True))
#model.add(LSTM(16, return_sequences=True))  # returns a sequence of vectors of dimension 16
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=learning_rate)
#model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=[keras.metrics.AUC(name='auc')])


history = model.fit(X_train_cnn,
                    y_train,
                    batch_size=batch_size,
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(X_test_cnn,y_test),
                    callbacks = [model_save, early_stop, reduce_lr],)

Epoch 1/15
1421/3127 [============>.................] - ETA: 9:58 - loss: 0.0015 - auc: 0.9096

In [ ]:
# Evaluation

In [ ]:
y_pred = model.predict(X_test_cnn, batch_size=512)
AUC = metrics.roc_auc_score(y_test,y_pred)
print("AUC: {:.3f}".format(AUC))

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('history.csv')

In [ ]:
### Training Curves

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history.epoch[-1]+1),history.history['val_auc'],label='val_auc')
plt.plot(range(history.epoch[-1]+1),history.history['auc'],label='auc')
plt.title('auc'); plt.xlabel('Epoch'); plt.ylabel('auc');plt.legend(); 
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history.epoch[-1]+1),history.history['val_loss'],label='Val_loss')
plt.plot(range(history.epoch[-1]+1),history.history['loss'],label='loss')
plt.title('loss'); plt.xlabel('Epoch'); plt.ylabel('loss');plt.legend(); 
plt.show()

In [ ]:
### ROC Curve

In [ ]:
def generate_results(y_test, y_score):
    # print(y_score)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
generate_results(y_test, y_pred)